In [1]:
from pyppeteer import launch
import asyncio
import nest_asyncio
from bs4 import BeautifulSoup as bs
import re
import time
import pandas as pd
import requests
import urllib.request

In [2]:
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)
pd.set_option('display.max_colwidth', 100)

nest_asyncio.apply()

In [23]:
async def get_item(categroy: str):
    # browser = await launch()
    print("Setting up broswer for web page...")
    
    browser = await launch({
            "headless": False,
        })
    page = await browser.newPage()
    
    await page.setViewport({'width': 1920, 'height': 1080})
    await page.goto(categroy, {'timeout': 10000*30})
    return page, browser

In [24]:
dir_path = "./Suning/"
p = pd.read_csv(f"{dir_path}Products_Suning.csv")
c = pd.read_csv(f"{dir_path}Customers_Suning.csv")

In [25]:
p

,Unnamed: 0,Product Index,Product Type,Product Name,Prodect Brand,Price,Total Comment,Seller,Website,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20,Unnamed: 21,Unnamed: 22,Unnamed: 23,Unnamed: 24
0,0,10964124069,Tablet,APPLE IPAD AIR 10.5 64GB WIFI GOLD MUUL2ZP/A 平板電腦,APPLE,3699.0,216,香港蘇寧,https://product.hksuning.com/0000000000/10964124069.html,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,10964145615,Tablet,APPLE IPAD MINI 2019 64GB WIFI SPACE GREY MUQW2ZP/A 平板電腦,APPLE,2988.0,203,香港蘇寧,https://product.hksuning.com/0000000000/10964145615.html,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,10555865475,Tablet,Apple iPad Pro 12.9 1TB Wi-Fi Space Gray MTFR2ZP/A 平板電腦,Apple,12499.0,5,香港蘇寧,https://product.hksuning.com/0000000000/10555865475.html,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,11359867696,Tablet,Apple iPad 10.2 Wifi 32GB Space Gray MW742ZP/A 平板電腦,Apple,2599.0,0,香港蘇寧,https://product.hksuning.com/0000000000/11359867696.html,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,10964145617,Tablet,APPLE IPAD MINI 2019 64GB WIFI SILVER MUQX2ZP/A 平板電腦,APPLE,2988.0,194,香港蘇寧,https://product.hksuning.com/0000000000/10964145617.html,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,5,11359867697,Tablet,Apple iPad 10.2 Wifi 32GB Silver MW752ZP/A 平板電腦,Apple,2599.0,0,香港蘇寧,https://product.hksuning.com/0000000000/11359867697.html,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,6,11359867698,Tablet,Apple iPad 10.2 Wifi 32GB Gold MW762ZP/A 平板電腦,Apple,2599.0,0,香港蘇寧,https://product.hksuning.com/0000000000/11359867698.html,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,7,11359867699,Tablet,Apple iPad 10.2 Wifi 128GB SpaceGray MW772ZP/A 平板電腦,Apple,3399.0,0,香港蘇寧,https://product.hksuning.com/0000000000/11359867699.html,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,8,11359867700,Tablet,Apple iPad 10.2 Wifi 128GB Silver MW782ZP/A 平板電腦,Apple,3399.0,0,香港蘇寧,https://product.hksuning.com/0000000000/11359867700.html,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,9,11359867701,Tablet,Apple iPad 10.2 WIifi 128GB Gold MW792ZP/A 平板電腦,Apple,3399.0,0,香港蘇寧,https://product.hksuning.com/0000000000/11359867701.html,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [21]:
filtered = p.sort_values(['total_comment'], ascending=False).groupby(['product_type']).head(9)

KeyError: 'total_comment'

In [15]:
filtered

NameError: name 'filtered' is not defined

In [16]:
async def get_imgs():
    for index, row in filtered.iterrows():
        print(f"Processing product {index}")
        p_type = row['product_type'].replace(' ', '_')
        p_name = row['product_name'].replace(' ', '_')
        p_sales = str(row['total_comment'])
        p_url = row['website']

        page, browser = asyncio.get_event_loop().run_until_complete(get_item(p_url))
        source = bs(await page.content(), 'lxml').prettify()

        img_regex = re.compile(r'src="(//imgservice.suning.cn/\S+/\S+/\S+/\S+.jpg_800w_800h_4e)"/>')
        img_url = re.findall(img_regex, source)[0]
        img_path = f"{dir_path}Top-Ten-Img/{p_type}-{p_sales}.jpg"

        urllib.request.urlretrieve(f"http:{img_url}", img_path)

        print(f"Saved image in {img_path}\n")

        await browser.close()


In [17]:
asyncio.get_event_loop().run_until_complete(get_imgs())

NameError: name 'filtered' is not defined